In [1]:
import pdfplumber
import re
import glob
import pandas as pd
import os

In [2]:
# a) Defina o diretório onde estão os PDFs
pdf_dir   = r'D:\Fausto Stangler\Downloads\bloco'
# b) Liste todos os arquivos .pdf nessa pasta
pdf_paths = glob.glob(os.path.join(pdf_dir, '*.pdf'))
pdf_paths

['D:\\Fausto Stangler\\Downloads\\bloco\\CENTRO CIRURGICO - SUS.pdf',
 'D:\\Fausto Stangler\\Downloads\\bloco\\escala 30 de abril - privado.pdf']

In [3]:
def text_from_pdf(path):
    """Retorna uma lista com cada linha de texto extraída do PDF."""
    lines = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                # splitlines preserva corretamente cada quebra de linha
                lines.extend(text.splitlines())
    return lines


In [138]:
content = []
for p in pdf_paths:
    content.append(text_from_pdf(p))
content

[['11 - HOSPITAL BANCO DE OLHOS ASSOCIACAO Página: 1 /11',
  'SOULMV - Sistema de Centro Cirúrgico e Obstétrico Emitido por: CIBELE.SILVA',
  'Relatório de Mapa Cirúrgico Em: 29/04/2025 17:48',
  'Data: 30/04/2025',
  'Centro Cirúrgico : 9 - CENTRO CIRURGICO - SUS',
  'Sala : 24 - SALA 1',
  'Hora Aviso Atend. Tipo Paciente Idade Telefone Leito UTI Rad. Prestador\\Atividade',
  '15:00 44293 Internação 320234 PERLA CHAVES ALTAMIRANDA 37 Não Não AMANDA SCHNORR',
  'COORE PARASTCHUK E VDS ANESTESIA',
  'Cirurgia Lateralidade Convênio / Plano Sub-Plano Status Autorização',
  '2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Esquerda SUS - INTERNACAO/PLANO UNICO Não Cadastrado',
  'VITRECTOMIA POSTERIOR COM INFUSAO DE PERFLUOCARBONO E E Esquerda SUS - INTERNACAO Não Cadastrado',
  'Tipo(s) de Anestesia: BLOQUEIO LOCAL + SEDACAO',
  '16:00 44294 Internação 322520 ERONI MORAIS DE OLIVEIRA 76 54 999510942 Não Não AMANDA SCHNORR',
  'COORE PARASTCHUK E VDS ANESTESIA',
  'Cirurgia Lat

In [140]:
def clean_content(content):
    """
    Limpa o conteúdo das páginas, removendo linhas que não pertencem à escala
    e separa as linhas de controle (Total de Avisos de Cirurgia).
    
    Args:
        content: lista de páginas, cada página é lista de linhas (strings).
    
    Returns:
        tuple:
            cleaned_content: mesma estrutura de páginas, sem as linhas de cabeçalho/rodapé
            control_lines: lista de todas as linhas de 'Total de Avisos de Cirurgia'
    """
    cleaned_content = []
    control_lines = []
    
    # Padrões de linhas a remover
    header_patterns = [
        r'^\d+\s*-\s*HOSPITAL',              # "11 - HOSPITAL BANCO..."
        r'^SOULMV',                          # "SOULMV - Sistema..."
        r'^Relatório de Mapa Cirúrgico',     # Título do relatório
        r'^GRUPO',                           # "GRUPO SÃO PIETRO..."
        r'^MV\s*\|',                         # "MV | SoulMV"
    ]
    # Padrão de linha de controle
    control_pattern = re.compile(r'^Total de Avisos de Cirurgia\s*:\s*\d+', re.IGNORECASE)
    
    for page in content:
        cleaned_page = []
        for line in page:
            line_stripped = line.strip()
            # Se for linha de controle, salva e não inclui no cleaned_page
            if control_pattern.match(line_stripped):
                control_lines.append(line_stripped)
                continue
            # Se bater qualquer header_pattern, descarta
            if any(re.match(pat, line_stripped, re.IGNORECASE) for pat in header_patterns):
                continue
            # Caso contrário, mantém na página limpa
            cleaned_page.append(line)
        cleaned_content.append(cleaned_page)
    
    return cleaned_content, control_lines

# Exemplo de uso:
content, controls = clean_content(content)

In [141]:
def extract_last_controls_per_doc(content):
    """
    Para cada documento (lista de páginas) em `contents`,
    retorna apenas o último 'Total de Avisos de Cirurgia'.
    """
    last_controls = []
    for pages in content:
        # usa clean_content para separar linhas de controle
        _, controls = clean_content(pages)
        if controls:
            last_controls.append(controls[-1])
    return last_controls

# Uso:
# supondo que content seja uma lista onde cada item é as páginas de um PDF
last_totals = extract_last_controls_per_doc(content)
print(last_totals)
# → ['Total de Avisos de Cirurgia : 32', 'Total de Avisos de Cirurgia : 41']









[]


In [115]:
def extract_data(page_lines):
    """
    Extrai a primeira data no formato 'DD/MM/YYYY' de uma página inteira.
    """
    for line in page_lines:
        m = re.match(r'\s*[Dd]ata\s*[:\-]\s*(\d{2}/\d{2}/\d{4})', line)
        if m:
            return m.group(1)
    return None


In [116]:
def extract_sala(line):
    """
    Se a linha começa com "Sala :", pega a última palavra,
    converte para int e retorna no formato "Sala X".
    """
    line = line.strip()
    if line.startswith("Sala :"):
        # ex: ["Sala", ":", "24", "-", "SALA", "1"] -> última palavra é "1"
        last = line.split()[-1]
        try:
            num = int(last)
            return f"Sala {num}"
        except ValueError:
            pass
    # Fallback genérico: captura "SALA X" em qualquer outro formato
    m = re.search(r"\bSALA\s*(\d+)\b", line, re.IGNORECASE)
    if m:
        num = int(m.group(1))
        return f"Sala {num}"
    return None


In [117]:
def extract_medico(lines):
    # 1) se houver rótulo explícito
    for line in lines:
        m = re.match(r'\s*[Mm]édico\s*[:\-]\s*(.+)', line)
        if m:
            return m.group(1).strip().title()

    # 2) pega a primeira linha que começa com horário
    record = next((l for l in lines if re.match(r'^\d{2}:\d{2}\s', l)), None)
    if record:
        # split na última ocorrência de 'Não' ou 'Sim'
        parts = re.split(r'\b(?:Não|Sim)\b', record, flags=re.IGNORECASE)
        name = parts[-1].strip()
        return name.title()

    return None


In [118]:
def extract_horario(lines):
    """
    Busca a primeira ocorrência de horário no formato 'HH:MM' em uma lista de linhas.
    """
    for line in lines:
        # Match de início de linha ou qualquer parte: HH:MM
        m = re.search(r'\b([01]\d|2[0-3]):([0-5]\d)\b', line)
        if m:
            return m.group(0)
    return None

In [124]:
def build_record(page_date, sala, lines):
    """
    Converte (sala, linhas) num dict chamando suas funções de extração.
    """
    return {
        'data'    : page_date,
        'horario'    : extract_horario(lines),
        'sala'       : sala,
        'medico'     : extract_medico(lines),
        # 'data'       : extract_data(lines),
        # 'cirurgia'   : extract_cirurgia(lines),
        # 'lateralidade': extract_lateralidade(lines),
        # 'anestesia'  : extract_anestesia(lines),
        # 'convenio'   : extract_convenio(lines),
        # 'observacao' : extract_observacao(lines),
    }


In [120]:
def parse_schedule(content):
    """
    Recebe content (lista de páginas, cada uma lista de linhas)
    e devolve uma lista de registros estruturados.
    """
    records = []

    for page in content:
        page_date = extract_data(page)
        current_sala = None
        current_block = None

        for line in page:
            # 1) atualiza a sala corrente
            sala = extract_sala(line)
            if sala:
                current_sala = sala
                continue

            # 2) nova cirurgia: linha que começa com HH:MM
            if re.match(r'^\d{2}:\d{2}\s', line):
                # salva o bloco anterior
                if current_block:
                    records.append(build_record(page_date, current_sala, current_block))
                # inicia novo bloco
                current_block = [line]
            else:
                # continua acumulando no bloco atual
                if current_block is not None:
                    current_block.append(line)

    # 3) salva o último bloco
    if current_block:
        records.append(build_record(page_date, current_sala, current_block))

    return records




In [125]:
# Uso:
escala = parse_schedule(content)
print(f"Total de cirurgias: {len(escala)}")
for r in escala:
    print(r)


Total de cirurgias: 72
{'data': '30/04/2025', 'horario': '15:00', 'sala': 'Sala 1', 'medico': 'Amanda Schnorr'}
{'data': '30/04/2025', 'horario': '16:00', 'sala': 'Sala 2', 'medico': 'Amanda Schnorr'}
{'data': '30/04/2025', 'horario': '15:30', 'sala': 'Sala 2', 'medico': 'Luiz Eduardo Osowski'}
{'data': '30/04/2025', 'horario': '16:00', 'sala': 'Sala 2', 'medico': 'Luiz Eduardo Osowski'}
{'data': '30/04/2025', 'horario': '16:30', 'sala': 'Sala 2', 'medico': 'Luiz Eduardo Osowski'}
{'data': '30/04/2025', 'horario': '17:00', 'sala': 'Sala 2', 'medico': 'Luiz Eduardo Osowski'}
{'data': '30/04/2025', 'horario': '17:30', 'sala': 'Sala 2', 'medico': 'Luiz Eduardo Osowski'}
{'data': '30/04/2025', 'horario': '18:00', 'sala': 'Sala 2', 'medico': 'Luiz Eduardo Osowski'}
{'data': '30/04/2025', 'horario': '18:30', 'sala': 'Sala 2', 'medico': 'Luiz Eduardo Osowski'}
{'data': '30/04/2025', 'horario': '19:00', 'sala': 'Sala 2', 'medico': 'Luiz Eduardo Osowski'}
{'data': '30/04/2025', 'horario': '19:3

In [132]:
def extract_cirurgia_segments(content):
    """
    Para cada página em `content` (lista de listas de linhas),
    retorna uma lista de listas com todas as linhas de cirurgia
    (entre 'Cirurgia...' e a próxima 'Observação...').
    """
    segments = []
    for page in content:
        capturing = False
        buffer = []
        for line in page:
            text = line.strip().lower()
            # Inicia captura após encontrar linha que começa com 'cirurgia'
            if text.startswith('cirurgia'):
                capturing = True
                buffer = []
                continue
            # Encerra captura quando encontrar 'observação' e salva o segmento
            if capturing and text.startswith('observação'):
                capturing = False
                if buffer:
                    segments.append(buffer[:])
                continue
            # Se estiver capturando, adiciona a linha completa
            if capturing:
                buffer.append(line)
    return segments

# Exemplo de uso:
cirurgias = extract_cirurgia_segments(content)
for idx, seg in enumerate(cirurgias, 1):
    for l in seg:
        print(l)

2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Esquerda SUS - AMBULATORIO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Esquerda SUS - AMBULATORIO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Esquerda SUS - AMBULATORIO/PLANO UNICO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Direita SUS - AMBULATORIO/PLANO UNICO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Direita SUS - AMBULATORIO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Direita SUS - AMBULATORIO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Esquerda SUS - AMBULATORIO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Direita SUS - AMBULATORIO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Direita SUS - AMBULATORIO Não Cadastrado
2 - FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-OCULAR DOB Esq